## Reading HTML with handles of US Senators

This notebook aims to gather the Twitter handles of all U.S. Senators, which will then be incorporated into a dataframe. Subsequently, the handles will be utilized to fetch the corresponding user_ids, as it is the user_id, not the username, that is essential for extracting tweets from each user's timeline.

Upon creating a dataset containing the user_ids of each Senator, our primary objective is to extract tweets from their respective accounts and reconstruct the Republican vs. Democrat dataset.

In [1]:
#import library
import pandas as pd

In [2]:
# read this webpage with handles from senators
members = pd.read_html("https://pressgallery.house.gov/member-data/members-official-twitter-handles")

In [3]:
# revise type of variable created
type(members)

list

Since it's a list, it is going to be complicated to deal with. We will have to convert into a DataFrame.

1.Save sublists into variables

In [4]:
# Saving each sublist (which represent a column) into a variable
member_name = members[0][0]
member_lastname =members[0][1]
member_handle =members[0][2]
member_party = members[0][4]

2. Create a dictionary with variables as values of each columns 

In [5]:
memb_dict = {'firstname' : member_name, 'last_name' : member_lastname, 'handle': member_handle, 'party': member_party}

3. Convert the dictionary into a DataFrame

In [6]:
members_data = pd.DataFrame(memb_dict)
members_data.head()

,firstname,last_name,handle,party
0,Last Updated:,1/22/2024,Subscribe to List: https://twitter.com/i/lists...,Subscribe to List: https://twitter.com/i/lists...
1,This list is updated as new handles are availa...,This list is updated as new handles are availa...,This list is updated as new handles are availa...,This list is updated as new handles are availa...
2,FirstName,LastName,Twitter Handle,Party
3,Alma,Adams,@RepAdams,D
4,Robert,Aderholt,@Robert_Aderholt,R


4. Clean the members_data2 dataframe since it has an erroneous row

In [7]:
# Delete the first row wich is an error
members_data2 = members_data.loc[3::,]

In [26]:
# Remove '@' from the 'handle' column
members_data2['handle'] = members_data2['handle'].str.lstrip('@')

C:\Users\erick\AppData\Local\Temp\ipykernel_10332\2700769160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  members_data2['handle'] = members_data2['handle'].str.lstrip('@')


In [28]:
# check results
members_data2.head()

,firstname,last_name,handle,party
3,Alma,Adams,RepAdams,D
4,Robert,Aderholt,Robert_Aderholt,R
5,Pete,Aguilar,RepPeteAguilar,D
6,Mark,Alford,RepMarkAlford,R
7,Rick,Allen,RepRickAllen,R


### Saving the Handles column into a list

In Twitter, in order to do a Timeline search using the Twitter API v2, one has to search by user_id and not username.

For this, we will use the dataframe created above to extract all of the handles into a list. This handles will be cleaned by eliminatin the " " and any space after each , since the query requires for usernames to be without the " " and with no space after the comas.

Also, there is a max amount of handles per query of 100, se we will divide the handles listo into 5 to make 5 queries. The user_id results will the be compiled together.

The user_ids will be used to search for tweets on each user's timeline.

In [29]:
# Creating list of handles
handles = members[0].loc[3::,2].str.replace("@","").to_list()

In [30]:
len(handles)

439

### Since Twitter API doesn't allow for more than 100 usernames to be queried, we will divide the list into lists of 99

In [31]:
handles1 = handles[:99]
len(handles1)

99

In [32]:
handles2 = handles[99:198]
len(handles2)

99

In [33]:
handles3 = handles[198:297]
len(handles3)

99

In [34]:
handles4 = handles[297:396]
len(handles4)

99

In [35]:
handles5 = handles[396:]
len(handles5)

43

In [36]:
len(handles1)+len(handles2)+len(handles3)+len(handles4)+len(handles5)

439

### Converting to plain text with GPT
We will convert each handles list to plain text without the " " and without  any spaces after the , .
Each plain text will be queried to obtain the User ID from each Username

In [37]:
# This line of code is used to extract each handles# list, convert it to plain text in GPT and query it on Postman.
handles5

['BennieGThompson',
 'CongressmanGT',
 'RepTiffany',
 'RepTimmons',
 'repdinatitus',
 'RepRashida',
 'RepJillTokuda',
 'RepPaulTonko',
 'NormaJTorres',
 'RepRitchie',
 'RepLoriTrahan',
 'RepDavidTrone',
 'RepMikeTurner',
 'RepUnderwood',
 'RepDavidValadao',
 'Congressman_JVD',
 'RepBethVanDuyne',
 'RepVanOrden',
 'RepJuanVargas',
 'RepGabeVasquez',
 'RepVeasey',
 'NydiaVelazquez',
 'RepAnnWagner',
 'RepWalberg',
 'RepWaltzPress',
 'RepDWStweets',
 'RepMaxineWaters',
 'RepBonnie',
 'TXRandy14',
 'RepWebster',
 'RepBradWenstrup',
 'RepWesterman',
 'RepWexton',
 'RepSusanWild',
 'RepNikema',
 'RepWilliams',
 'RepRWilliams',
 'RepWilson',
 'RepJoeWilson',
 'RobWittman',
 'rep_stevewomack',
 'RepRudyYakym',
 'RepRyanZinke']

### Joining JSON files
Each query returns a JSON file. The 5 JSON files with the user_id's will be joined into one CSV file

In [38]:
import json

# Load the JSON file
file_path = "handles/complete_handles.json"  # Replace with the actual path
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract the list of dictionaries from the 'data' key
data_list = data.get('data', [])

# Create a DataFrame
df = pd.DataFrame(data_list, columns=['id', 'name', 'username', 'party'])

# Save the DataFrame to a CSV file
csv_file_path = "handles/complete_handles_dataframe.csv"  # Replace with the desired path
df.to_csv(csv_file_path, index=False)

print("DataFrame \'complete_handles_dataframe\" created and saved to CSV successfully!")


DataFrame 'complete_handles_dataframe" created and saved to CSV successfully!


### This new CSV file will be read into a DatFrame to map the Party with the correct value on the members_data2.csv file created above

In [39]:
# Read new CSV into a Df
df = pd.read_csv('handles/complete_handles_dataframe.csv')
df.head()

,id,name,username,party
0,2916086925,Rep. Alma Adams,RepAdams,NaN
1,76452765,Robert Aderholt,Robert_Aderholt,NaN
2,3018670151,Rep. Pete Aguilar,RepPeteAguilar,NaN
3,1612483604071727104,Mark Alford,RepMarkAlford,NaN
4,2964287128,Rick W. Allen,RepRickAllen,NaN


Mapping the username on the df DataFrame with the handle in the members_data2 DataFrame to obtain the Party for that user

In [40]:
# Iterate through each row in df
for i in range(len(df)):
    username = df.loc[i, 'username']
    matching_row = members_data2[members_data2['handle'] == username]
    
    if not matching_row.empty:
        df.loc[i, 'party'] = matching_row['party'].iloc[0]
    else:
        # Handle the case where there is no matching row in members_data2
        df.loc[i, 'party'] = None  # or any other value indicating no match

In [41]:
#Display Results
df

,id,name,username,party
0,2916086925,Rep. Alma Adams,RepAdams,D
1,76452765,Robert Aderholt,Robert_Aderholt,R
2,3018670151,Rep. Pete Aguilar,RepPeteAguilar,D
3,1612483604071727104,Mark Alford,RepMarkAlford,R
4,2964287128,Rick W. Allen,RepRickAllen,R
...,...,...,...,...
421,234014087,Rep. Frederica Wilson,RepWilson,D
422,254082173,Joe Wilson,RepJoeWilson,R
423,15356407,Rep. Rob Wittman,RobWittman,R
424,234469322,Rep. Steve Womack,rep_stevewomack,R


In [42]:
# Save into a new file
df.to_csv('PoliticalUsernamesAndIds.csv', index=False)

### Convert JSON file with all tweets to csv

This step is performed after scraping all tweets in the Tweet_extractor.ipynb file and compiling the results inthe json_files.compiler.ipynb file

In [ ]:
# Load JSON data from a file
with open('tweet_dataset/RepublicanVsDemocratNew.json') as json_file:
    data = json.load(json_file)

# Convert JSON data to a DataFrame
df = pd.json_normalize(data)

# Export DataFrame to a CSV file
df.to_csv('RepublicanVsDemocrat.csv', index=False)

In [ ]:
# read into a df to check results
repvsdem = pd.read_csv('RepublicanVsDemocrat.csv')

repvsdem.head()
print(len(repvsdem))

We can see it is now saved into a csv file successfully. The only detail is that there is only one row for each username, and all tweets per username are saved into a list. We will address this on the erick_rudelman_capstone.ipynb file

### Conclusion

We have successfully generated a dataframe containing the required user_id information for extracting tweets from the timelines of all U.S. senators. Additionally, the "PoliticalUsernamesAndIds.csv" file correlates the Party affiliation values to each senator using the members_data2 dataframe, which was created from the list of U.S. Senator handles obtained from https://pressgallery.house.gov/member-data/members-official-twitter-handles.

The tweet extractor code will be executed on the 'Tweet_extracot.ipynb' file.